In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import geopandas
import numpy as np
from shapely.geometry import Point
import math
import glob

#Generate inline plots
%matplotlib inline

files = [f for f in glob.glob("./data/csse_*.csv")]

dfs = []
columns = ['Province/State','Country/Region','Last Update','Confirmed']
for f in files:
    df = pd.read_csv(f,parse_dates=['Last Update'],usecols=columns)
    dfs.append(df)
    
    
df = pd.concat(dfs)
df = df.set_index('Last Update')
df

,Province/State,Country/Region,Confirmed
Last Update,,,
2020-01-26 23:00:00,Hubei,Mainland China,1423.0
2020-01-26 23:00:00,Guangdong,Mainland China,146.0
2020-01-26 23:00:00,Zhejiang,Mainland China,128.0
2020-01-26 23:00:00,Henan,Mainland China,128.0
2020-01-26 23:00:00,Chongqing,Mainland China,110.0
...,...,...,...
2020-01-27 19:00:00,NaN,Cambodia,1.0
2020-01-27 19:00:00,NaN,Sri Lanka,1.0
2020-01-27 19:00:00,NaN,Ivory Coast,1.0


In [37]:
def create_location(row):
    if pd.isna(row['Province/State']):
        return row['Country/Region']
    else:
        return row['Province/State'] + ', ' + row['Country/Region']

df['Location'] = df.apply(create_location, axis=1)
    
daily_confirmed = df.groupby('Location').resample('D')['Confirmed'].mean()
daily_confirmed

Location                  Last Update
Anhui, Mainland China     2020-01-21       NaN
                          2020-01-22       1.0
                          2020-01-23       9.0
                          2020-01-24      15.0
                          2020-01-25      46.0
                                         ...  
Zhejiang, Mainland China  2020-01-24      43.0
                          2020-01-25      76.0
                          2020-01-26     116.0
                          2020-01-27     143.0
                          2020-01-28     173.0
Name: Confirmed, Length: 350, dtype: float64